In [1]:
#import necessary packages
import pandas as pd
import numpy as np

In [15]:
quotes = [1.57, 3.9, 7.5]
result = []
res = []
sum = 0
count = 0
for i in quotes:
    tmp = 1/i
    #print(tmp)
    sum = sum + tmp
    result.append(tmp)
    count = count +1
sum2 = 0
for i in result:
    tmp = i/sum
    sum2 = sum2 + tmp
    res.append(tmp)
print(res)
print(sum2)

[0.6203868668872486, 0.24974548231102064, 0.1298676508017307]
1.0


In [24]:
PL_20_21 = pd.read_csv('Data/2020-21/Fixtures/E_20_21.csv')
PL_20_21['B365H'] = 1/PL_20_21['B365H']
PL_20_21['B365D'] = 1/PL_20_21['B365D']
PL_20_21['B365A'] = 1/PL_20_21['B365A']
PL_20_21['SUM'] = PL_20_21['B365H'] + PL_20_21['B365D'] + PL_20_21['B365A']
PL_20_21['B365H'] = PL_20_21['B365H']/PL_20_21['SUM']
PL_20_21['B365D'] = PL_20_21['B365D']/PL_20_21['SUM']
PL_20_21['B365A'] = PL_20_21['B365A']/PL_20_21['SUM']
PL_20_21

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,SUM
0,E0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,...,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87,1.051208
1,E0,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12,1.052214
2,E0,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,...,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04,1.053180
3,E0,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,...,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86,1.053352
4,E0,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,...,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97,1.053756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,23/05/2021,16:00,Liverpool,Crystal Palace,2,0,H,1,0,...,-2.25,1.86,2.04,1.88,2.03,1.98,2.14,1.88,2.00,1.061507
376,E0,23/05/2021,16:00,Man City,Everton,5,0,H,2,0,...,-1.75,2.01,1.89,1.99,1.89,2.20,2.00,2.03,1.85,1.048291
377,E0,23/05/2021,16:00,Sheffield United,Burnley,1,0,H,1,0,...,0.00,2.04,1.86,2.05,1.86,2.17,1.90,2.03,1.84,1.051587
378,E0,23/05/2021,16:00,West Ham,Southampton,3,0,H,2,0,...,-0.75,2.00,1.90,2.02,1.91,2.06,2.01,1.99,1.89,1.056061
